In [1]:
# 3.1 神经网路基本组成
# 空洞卷积
from torch import nn
# 定义普通卷积 dilation=1
conv1 = nn.Conv2d(3, 256, 3, stride=1, padding=1, dilation=1)
conv1

Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [2]:
# 定义空洞卷积
conv2 = nn.Conv2d(3, 356, 3, stride=1, padding=1, dilation=2)
conv2

Conv2d(3, 356, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(2, 2))

In [3]:
# 3.2 走向深度
from torch import nn
class VGG(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG, self).__init__()
        layers = []
        in_dim = 3
        out_dim = 64
        for i in range(13):
            layers += [nn.Conv2d(in_dim, out_dim, 3, 1, 1), nn.ReLU(inplace=True)]
            in_dim = out_dim
            if i == 1 or i == 3 or i == 6 or i == 9 or i == 12:
                layers += [nn.MaxPool2d(2, 2)]
                if i != 9:
                    out_dim *= 2
        self.features = nn.Sequential(*layers)
        self.classifier = nn.Sequential(nn.Linear(512 * 7 * 7, 4096), 
                                        nn.ReLU(True), nn.Dropout(), 
                                        nn.Linear(4096, 4096), 
                                        nn.ReLU(True), 
                                        nn.Dropout(), 
                                        nn.Linear(4096, num_classes), 
                                       )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [4]:
import torch
vgg = VGG(21).cuda()
input = torch.randn(1, 3, 224, 224).cuda()
input.shape

torch.Size([1, 3, 224, 224])

In [5]:
scores = vgg(input)
scores.shape

torch.Size([1, 21])

In [6]:
features = vgg.features(input)
features.shape

torch.Size([1, 512, 7, 7])

In [7]:
vgg.features, vgg.classifier

(Sequential(
   (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU(inplace=True)
   (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU(inplace=True)
   (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (6): ReLU(inplace=True)
   (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (8): ReLU(inplace=True)
   (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (11): ReLU(inplace=True)
   (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (13): ReLU(inplace=True)
   (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (15): ReLU(inplace=True)
   (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (17): Conv2d(

In [9]:
# 3.3 纵横交错
import torch
from torch import nn
import torch.nn.functional as F

In [10]:
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding)
    def forward(self, x):
        x = self.conv(x)
        return F.relu(x, inplace=True)

In [14]:
a = torch.tensor([[1, 2, 3], 
                  [4, 5, 6]])
b = torch.tensor([[2, 3, 4], 
                  [5, 6, 7]])
torch.cat((a, b), 0), torch.cat((a, b), 1)

(tensor([[1, 2, 3],
         [4, 5, 6],
         [2, 3, 4],
         [5, 6, 7]]),
 tensor([[1, 2, 3, 2, 3, 4],
         [4, 5, 6, 5, 6, 7]]))

In [16]:
torch.stack((a, b), 0), torch.stack((a, b), 1)

(tensor([[[1, 2, 3],
          [4, 5, 6]],
 
         [[2, 3, 4],
          [5, 6, 7]]]),
 tensor([[[1, 2, 3],
          [2, 3, 4]],
 
         [[4, 5, 6],
          [5, 6, 7]]]))

In [36]:
class Inceptionv1(nn.Module):
    def __init__(self, in_dim,
                 hid_1_1, 
                 hid_2_1, hid_2_3, 
                 hid_3_1, hid_3_5, 
                 hid_4_1):
        super(Inceptionv1, self).__init__()
        self.branch_1 = BasicConv2d(in_dim, hid_1_1, 1)
        self.branch_2 = nn.Sequential(BasicConv2d(in_dim, hid_2_1, 1), 
                                      BasicConv2d(hid_2_1, hid_2_3, 3, padding=1))
        self.branch_3 = nn.Sequential(BasicConv2d(in_dim, hid_3_1, 1), 
                                      BasicConv2d(hid_3_1, hid_3_5, 5, padding=2))
        self.branch_4 = nn.Sequential(nn.MaxPool2d(3, stride=1, padding=1), 
                                      BasicConv2d(in_dim, hid_4_1, 1))
    def forward(self, x):
        b1 = self.branch_1(x)
        b2 = self.branch_2(x)
        b3 = self.branch_3(x)
        b4 = self.branch_4(x)
        output = torch.cat((b1, b2, b3, b4), dim=1)
        return output

In [37]:
net_inceptionv1 = Inceptionv1(3, 64, 32, 64, 64, 96, 32)
net_inceptionv1

Inceptionv1(
  (branch_1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (branch_2): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (branch_3): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): BasicConv2d(
      (conv): Conv2d(64, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
  )
  (branch_4): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (1): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    )
  )
)

In [38]:
input_1 = torch.randn(1, 3, 256, 256)
input_1.size()

torch.Size([1, 3, 256, 256])

In [39]:
output = net_inceptionv1(input_1)

In [40]:
output.shape

torch.Size([1, 256, 256, 256])

In [50]:
# 3.4 ResNet
import torch
from torch import nn
class Bottleneck(nn.Module):
    def __init__(self, in_dim, out_dim, stride=1):
        super(Bottleneck, self).__init__()
        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_dim, in_dim, 1, bias=False), nn.BatchNorm2d(in_dim), nn.ReLU(inplace=True), 
            nn.Conv2d(in_dim, in_dim, 3, stride, 1, bias=False), nn.BatchNorm2d(in_dim), nn.ReLU(inplace=True), 
            nn.Conv2d(in_dim, out_dim, 1, bias=False), nn.BatchNorm2d(out_dim),)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, 1, 1), 
            nn.BatchNorm2d(out_dim),)
    def forward(self, x):
        identity = x
        out = self.bottleneck(x)
        identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

In [51]:
bottleneck_1_1 = Bottleneck(64, 256)
bottleneck_1_1

Bottleneck(
  (bottleneck): Sequential(
    (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (relu): ReLU(inplace=True)
  (downsample): Sequential(
    (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [52]:
input_1 = torch.randn(1, 64, 56, 56)
output = bottleneck_1_1(input_1)
output.shape

torch.Size([1, 256, 56, 56])

In [1]:
# 3.5 DenseNet
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
# Bottleneck 
class Bottleneck(nn.Module):
    def __init__(self, nChannels, growthRate):
        super(Bottleneck, self).__init__()
        interChannels = 4 * growthRate
        self.bn1 = nn.BatchNorm2d(nChannels)
        self.conv1 = nn.Conv2d(nChannels, interChannels, kernel_size=1, bias=False)
        self.bn2 = nn.BatchNorm2d(interChannels)
        self.conv2 = nn.Conv2d(interChannels, growthRate, kernel_size=3, padding=1, bias=False)
    def forward(self, x):
        out = self.conv1(F.relu(self.bn1(x)))
        out = self.conv2(F.relu(self.bn2(out)))
        out = torch.cat((x, out), 1)
        return out

In [7]:
class Denseblock(nn.Module):
    def __init__(self, nChannels, growthRate, nDenseBlocks):
        super(Denseblock, self).__init__()
        layers = []
        for i in range(int(nDenseBlocks)):
            layers.append(Bottleneck(nChannels, growthRate))
            nChannels += growthRate
        self.denseblock = nn.Sequential(*layers)
    def forward(self, x):
        return self.denseblock(x)

In [8]:
import torch
denseblock = Denseblock(64, 32, 6).cuda()
print(denseblock)

Denseblock(
  (denseblock): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): Bottleneck(
      (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (2): Bottleneck(
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=F

In [9]:
input_1 = torch.randn(1, 64, 256, 256).cuda()
output = denseblock(input_1)
output.shape

torch.Size([1, 256, 256, 256])